In [1]:
from datasets import load_dataset

dataset = load_dataset("tapaco", "ru")

Reusing dataset tapaco (/home/merionum/.cache/huggingface/datasets/tapaco/ru/1.0.0/71d200534b520a174927a8f0479c06220a0a6fb5201a84ebfce19006c6354698)


  0%|          | 0/1 [00:00<?, ?it/s]

In [29]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd


model_checkpoint = 'cointegrated/rubert-base-cased-nli-twoway'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
if torch.cuda.is_available():
    model.cuda()

def predict_pair(text1, text2):
    with torch.inference_mode():
        out = model(**tokenizer(text1, text2, return_tensors='pt').to(model.device))
        proba = torch.softmax(out.logits, -1).cpu().numpy()[0]
        return {v: proba[k] for k, v in model.config.id2label.items()}['entailment']

In [18]:
import pandas as pd

df = pd.DataFrame(dataset['train'])

In [30]:
df = df.groupby('paraphrase_set_id').agg({'paraphrase': list}).reset_index()

In [62]:
import tqdm
from itertools import combinations

data = []
for _, row in tqdm.tqdm(df.iterrows(), total=len(df)):
    cluster_id = row['paraphrase_set_id']
    pairs = combinations(row['paraphrase'], 2)
    for t1, t2 in pairs:
        ent1 = predict_pair(t1, t2)
        ent2 = predict_pair(t2, t1)
        data.append({
            "cluster_id": cluster_id,
            "text1": t1,
            "text2": t2,
            "ent1": ent1,
            "ent2": ent2
        })

 88%|████████▊ | 80251/91240 [57:43<03:29, 52.39it/s]   IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [66]:
df

,paraphrase_set_id,paraphrase
0,1,"[Я ем сыр., Я ем йогурт., Я кушаю сыр., Я съел..."
1,10,"[Он говорит по-английски?, Вы говорите по-англ..."
2,100,"[Ты сегодня вечером смотрел телевизор?, Ты сег..."
3,10002,"[Линкольн — важная персона., Линкольн — важный..."
4,100024,"[Откуда у неё этот список?, Где она взяла этот..."
...,...,...
91235,9997,"[Я оставил свою карточку дома., Я забыл свою к..."
91236,9998,"[У тебя есть кредитная карточка?, У тебя есть ..."
91237,99981,"[Где ты научился карате?, Где ты научилась кар..."
91238,9999,"[Я буду бороться до смерти., Я буду сражаться ..."


In [71]:
# df_ent.to_csv('data/tapaco_entailment.csv', index=False)

In [3]:
import pandas as pd

df = pd.read_csv('data/tapaco_entailment.csv')

In [8]:
df = df[(df.ent1 > 0.7) & (df.ent2 > 0.7)]

In [11]:
df = df[~(df.text1.str.lower().str.contains('чёрт|черт|бля|хрен|целк|хер|чё |че') | df.text2.str.lower().str.contains('чёрт|черт|бля|хрен|целк|хер|чё |че'))]

In [14]:
df = df[(df.text1.str.len() + df.text2.str.len()) > 35]

In [16]:
df = df[(df.text1.str.split().apply(len) > 3) & (df.text2.str.split().apply(len) > 3)]

In [18]:
df = df[~(df.text1.apply(lambda x: '...' in x)) | (df.text2.apply(lambda x: '...' in x))]

In [20]:
def vity(row):
    if 'вы' in row['text1'].lower() and 'ты' in row['text2'].lower():
        return True
    elif 'вы' in row['text2'].lower() and 'ты' in row['text1'].lower():
        return True
    return False

In [21]:
df = df[~df.apply(vity, axis=1)]

In [22]:
from alphabet_detector import AlphabetDetector
ad = AlphabetDetector()


df = df[~(df.text1.apply(lambda x: ad.is_latin(x)))]
df = df[~(df.text2.apply(lambda x: ad.is_latin(x)))]

In [24]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [25]:
df['leven'] = df.apply(lambda x: similar(x['text1'], x['text2']), axis=1)

In [26]:
df

,cluster_id,text1,text2,ent1,ent2,leven
90,10,Говорите ли Вы по-английски?,Вы можете говорить по-английски?,0.944408,0.929248,0.666667
96,10,Говорите ли Вы по-английски?,Вы разговариваете на английском?,0.941382,0.930654,0.566667
115,10,Вы разве не говорите по-английски?,Вы можете говорить по-английски?,0.936903,0.764817,0.818182
121,10,Вы разве не говорите по-английски?,Вы разговариваете на английском?,0.934022,0.708513,0.606061
142,10,Вы можете говорить по-английски?,Вы разговариваете на английском?,0.929985,0.937610,0.562500
...,...,...,...,...,...,...
408736,9997,Я забыла дома кредитку.,Я оставил дома свою кредитную карточку.,0.947373,0.928390,0.612903
408737,9997,Я забыла дома кредитку.,Я оставил кредитную карточку дома.,0.888700,0.925407,0.491228
408738,9997,Я оставил дома свою кредитную карточку.,Я оставил кредитную карточку дома.,0.876875,0.949827,0.794521
408739,9998,У тебя есть кредитная карточка?,У тебя есть кредитка?,0.977508,0.951577,0.807692


In [30]:
df = df[df['leven'] < 0.9]

In [31]:
from razdel import tokenize

sw = ('ну', 'хуй', 'хуя', 'ээ', 'эээ', 'мэм',
      'ох', 'ах', 'вау', 'бог', 'боже', 'эй', '...', '—', 'дерьмо')
def stopwords(s):
    for tok in tokenize(s):
        if tok.text.lower() in sw:
            return True
    return False

def repeat(s):
    toks = [_.text for _ in tokenize(s)]
    if len(toks) / len(set(toks)) > 2:
        return True
    return False

In [32]:
df = df[~df.text1.apply(stopwords)]
df = df[~df.text2.apply(stopwords)]

In [34]:
df = df[~df.text1.apply(repeat)]
df = df[~df.text2.apply(repeat)]

In [44]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)
from collections import Counter
from razdel import tokenize

segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

def match_ents(s1, s2):
    doc1, doc2 = Doc(s1), Doc(s2)
    
    doc1.segment(segmenter)
    doc2.segment(segmenter)
    
    doc1.tag_ner(ner_tagger)
    doc2.tag_ner(ner_tagger)
#     print(doc1.spans, doc2.spans)
    if Counter([s.type for s in doc1.spans]) != Counter([s.type for s in doc2.spans]):
        return False
    
    numbers1 = set()
    numbers2 = set()
    
    for tok in doc1.tokens:
        if has_numbers(tok.text):
            numbers1.add(tok.text)
    
    for tok in doc2.tokens:
        if has_numbers(tok.text):
            numbers2.add(tok.text)
    
    if numbers1 != numbers2:
        return False
    
    return True
    

def has_numbers(inputString):
    return any(char.isdigit() for char in inputString)

def match_abbr(s1, s2):
    abbrs1 = set()
    abbrs2 = set()
    
    for tok in tokenize(s1):
        if tok.text.isupper() and len(tok.text) > 1:
            abbrs1.add(tok.text)
    for tok in tokenize(s2):
        if tok.text.isupper() and len(tok.text) > 1:
            abbrs2.add(tok.text)
    if abbrs1 == abbrs2:
        return True
    return False

def match_latin(s1, s2):
    lat1 = set()
    lat2 = set()
    
    for tok in tokenize(s1):
        if ad.is_latin(tok.text) and tok.text.isalpha():
            lat1.add(tok.text)
    for tok in tokenize(s2):
        if ad.is_latin(tok.text) and tok.text.isalpha():
            lat2.add(tok.text)

    if lat1 == lat2:
        return True
    return False

In [37]:
df = df[df.apply(lambda x: match_latin(x['text1'], x['text2']), axis=1)]

In [39]:
df = df[~df.text1.apply(lambda x: 'блин' in x)]
df = df[~df.text2.apply(lambda x: 'блин' in x)]

In [41]:
df = df[df.apply(lambda x: match_abbr(x['text1'], x['text2']), axis=1)]

In [45]:
import tqdm

ner_data = []
for i, row in tqdm.tqdm(df.iterrows(), total=len(df)):
    if match_ents(row['text1'], row['text2']):
        ner_data.append(i)

100%|██████████| 67523/67523 [02:18<00:00, 488.80it/s]


In [50]:
df = df[df.index.isin(ner_data)]

In [51]:
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)

In [52]:
def root_gender(s1, s2):
    try:
        g = set()
        d1 = Doc(s1)
        d1.segment(segmenter)
        d1.parse_syntax(syntax_parser)
        d1.tag_morph(morph_tagger)

        d2 = Doc(s2)
        d2.segment(segmenter)
        d2.parse_syntax(syntax_parser)
        d2.tag_morph(morph_tagger)

        root_g1 = g.add([_.feats['Gender'] for _ in d1.tokens if _.rel =='root'][0])
        root_g2 = g.add([_.feats['Gender'] for _ in d2.tokens if _.rel =='root'][0])
    #     print(g)
        if g == {'Fem', 'Masc'}:
            return False
    except:
        pass
    return True

In [53]:
gen_data = []

for i, row in tqdm.tqdm(df.iterrows(), total=len(df)):
    if root_gender(row['text1'], row['text2']):
        gen_data.append(i)

100%|██████████| 64796/64796 [04:23<00:00, 245.49it/s]


In [55]:
df = df[df.index.isin(gen_data)]

In [56]:
from collections import Counter

c = Counter()

for _, row in tqdm.tqdm(df.iterrows(), total=len(df)):
    c.update(Counter(row['text1']))
    c.update(Counter(row['text2']))

100%|██████████| 61702/61702 [00:03<00:00, 15431.57it/s]


In [ ]:
настоящее vs прошедшее время
он vs вы
то же самое что в opus

In [62]:
len(df)

61702

In [91]:
def same(s1, s2):
    toks1 = set([_.text for _ in tokenize(s1) if _.text.isalpha()])
    toks2 = set([_.text for _ in tokenize(s2) if _.text.isalpha()])
    
    if toks1 == toks2:
        return True
    return False

In [92]:
df = df[~df.apply(lambda x: same(x['text1'], x['text2']), axis=1)]

In [93]:
len(df)

58813

In [109]:
ty = ('ты', 'тобой', 'тебе', 'тебя')
vy = ('вы', 'вас', 'вами', 'вам')

In [112]:
vyty_id = df[((df.text1.str.contains('|'.join(ty))) & (df.text2.str.contains('|'.join(vy)))) |
   ((df.text1.str.contains('|'.join(vy))) & (df.text2.str.contains('|'.join(ty)))
   
   )].index

In [117]:
df = df[~df.index.isin(vyty_id)]

In [119]:
df.to_csv('data/clean/tapaco_clean.tsv', sep='\t', index=False)

In [120]:
df

,cluster_id,text1,text2,ent1,ent2,leven
90,10,Говорите ли Вы по-английски?,Вы можете говорить по-английски?,0.944408,0.929248,0.666667
96,10,Говорите ли Вы по-английски?,Вы разговариваете на английском?,0.941382,0.930654,0.566667
115,10,Вы разве не говорите по-английски?,Вы можете говорить по-английски?,0.936903,0.764817,0.818182
121,10,Вы разве не говорите по-английски?,Вы разговариваете на английском?,0.934022,0.708513,0.606061
142,10,Вы можете говорить по-английски?,Вы разговариваете на английском?,0.929985,0.937610,0.562500
...,...,...,...,...,...,...
408732,9997,Я забыл свою кредитку дома.,Я оставил кредитную карточку дома.,0.956427,0.916173,0.622951
408734,9997,Я забыл свою кредитную карточку дома.,Я оставил дома свою кредитную карточку.,0.973596,0.946152,0.763158
408735,9997,Я забыл свою кредитную карточку дома.,Я оставил кредитную карточку дома.,0.938201,0.943098,0.816901
408738,9997,Я оставил дома свою кредитную карточку.,Я оставил кредитную карточку дома.,0.876875,0.949827,0.794521


In [106]:
a = df.sample(1).iloc[0]

print(a['text1'])
print(a['text2'])

Ад выстлан добрыми намерениями.
Благими намерениями дорожка в ад вымощена.
